<a href="https://colab.research.google.com/github/DhanushSM/DhanushSM/blob/main/Enhanced_Heart_Disease_Prediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import numpy as np
from termcolor import colored

# --- Scikit-learn Imports ---
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# Metrics
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay, precision_score, recall_score, f1_score

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

#################################################################
# 1. ENHANCED DATA PROCESSOR CLASS
#################################################################

class EnhancedDataProcessor:
    """
    Enhanced data processor with better validation and quality checks.
    """
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = None
        self.preprocessor = None
        # Original column names -> Friendly names
        self.column_mapping = {
            'cp': 'chest_pain_type',
            'trestbps': 'resting_blood_pressure',
            'chol': 'cholesterol',
            'fbs': 'fasting_blood_sugar',
            'restecg': 'resting_ecg',
            'thalach': 'max_heart_rate',
            'exang': 'exercise_angina',
            'ca': 'vessels_colored',
            'thal': 'thalassemia'
        }

    def load_data(self):
        print("  Loading data...")
        try:
            self.data = pd.read_csv(self.filepath)
            print(colored(f"  Data loaded successfully. Shape: {self.data.shape}", "green"))
        except FileNotFoundError:
            print(colored(f"Error: File not found at {self.filepath}", "red"))
            self.data = None
        except Exception as e:
            print(colored(f"Error loading data: {e}", "red"))
            self.data = None

    def validate_data_quality(self, target_variable="target"):
        """Enhanced data validation with comprehensive checks"""
        if self.data is None:
            return False

        print("\n--- Data Quality Check ---")

        # Check for missing values
        missing_values = self.data.isnull().sum()
        if missing_values.sum() > 0:
            print(colored(f"  Missing values found:", "yellow"))
            for col, count in missing_values[missing_values > 0].items():
                print(f"    {col}: {count} missing values")
        else:
            print("  No missing values detected ✓")

        # Check target variable distribution
        if target_variable in self.data.columns:
            target_dist = self.data[target_variable].value_counts()
            print(f"  Target distribution:\n{target_dist}")
            if len(target_dist) == 2:
                ratio = target_dist[1] / target_dist[0]
                print(f"  Class ratio: {ratio:.2f}:1")

                if target_dist.min() < len(self.data) * 0.3:
                    print(colored("  ⚠️  Significant class imbalance detected!", "yellow"))
                else:
                    print("  Balanced dataset ✓")

        # Check for outliers in numerical features
        numerical_cols = ['age', 'resting_blood_pressure', 'cholesterol', 'max_heart_rate', 'oldpeak']
        numerical_cols = [col for col in numerical_cols if col in self.data.columns]

        print("  Outlier detection:")
        for col in numerical_cols:
            Q1 = self.data[col].quantile(0.25)
            Q3 = self.data[col].quantile(0.75)
            IQR = Q3 - Q1
            outliers = ((self.data[col] < (Q1 - 1.5 * IQR)) |
                       (self.data[col] > (Q3 + 1.5 * IQR))).sum()
            if outliers > 0:
                print(f"    {col}: {outliers} outliers detected")
            else:
                print(f"    {col}: No outliers ✓")

        return True

    def rename_columns(self):
        if self.data is not None:
            self.data.rename(columns=self.column_mapping, inplace=True)
            print("  Columns renamed for clarity.")

    def drop_duplicates(self):
        if self.data is not None:
            rows_dropped = len(self.data)
            self.data.drop_duplicates(inplace=True)
            rows_dropped -= len(self.data)
            if rows_dropped > 0:
                print(f"  Dropped {rows_dropped} duplicate rows.")
            else:
                print("  No duplicate rows found ✓")

    def handle_missing_values(self):
        if self.data is not None:
            rows_before = len(self.data)
            self.data.dropna(inplace=True)
            rows_after = len(self.data)
            rows_dropped = rows_before - rows_after
            if rows_dropped > 0:
                print(f"  Dropped {rows_dropped} rows with missing values.")
            else:
                print("  No rows with missing values to drop ✓")

    def build_preprocessor(self, numerical_cols, categorical_cols):
        """
        Builds a ColumnTransformer to handle all preprocessing.
        """
        print("  Building data preprocessor...")

        # Numerical pipeline: Impute with median, then scale
        numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])

        # Categorical pipeline: Impute with most frequent, then one-hot encode
        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ])

        # Combine transformers
        self.preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numerical_cols),
                ('cat', categorical_transformer, categorical_cols)
            ],
            remainder='passthrough'
        )
        print(colored("  Preprocessor built successfully.", "green"))

    def process_training_data(self, X):
        """
        Fits the preprocessor to the training data and transforms it.
        """
        if self.preprocessor is None:
            print(colored("Error: Preprocessor not built yet.", "red"))
            return None

        print("  Fitting and transforming training data...")
        X_processed = self.preprocessor.fit_transform(X)
        feature_names = self.preprocessor.get_feature_names_out()
        print(f"  Training data processed. New shape: {X_processed.shape}")
        print(f"  Number of features after preprocessing: {len(feature_names)}")
        return X_processed

    def save_preprocessor(self, filepath="enhanced_data_preprocessor.pkl"):
        """Saves the fitted preprocessor to a file."""
        if self.preprocessor:
            joblib.dump(self.preprocessor, filepath)
            print(colored(f"  Data preprocessor saved to {filepath}", "cyan"))
        else:
            print(colored("Error: No preprocessor to save.", "red"))

#################################################################
# 2. ENHANCED MODEL TRAINER CLASS
#################################################################

class EnhancedModelTrainer:
    """
    Enhanced model trainer with cross-validation and comprehensive evaluation.
    """
    def __init__(self):
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None

        # Enhanced models with tuned parameters
        self.models = {
            'Logistic Regression': LogisticRegression(
                random_state=42, max_iter=2000, class_weight='balanced',
                C=0.1, solver='liblinear'
            ),
            'Random Forest': RandomForestClassifier(
                random_state=42, class_weight='balanced',
                n_estimators=200, max_depth=10, min_samples_split=5
            ),
            'Support Vector Machine': SVC(
                random_state=42, probability=True, class_weight='balanced',
                C=1.0, kernel='rbf', gamma='scale'
            ),
            'Gradient Boosting': GradientBoostingClassifier(
                random_state=42, n_estimators=100, max_depth=3,
                learning_rate=0.1, subsample=0.8
            )
        }
        self.results = {}
        self.trained_models = {}
        self.cv_results = {}

    def perform_cross_validation(self, X, y, cv_folds=5):
        """Perform cross-validation for better model evaluation"""
        print(f"\n--- Performing {cv_folds}-fold Cross-Validation ---")
        cv_results = {}

        for name, model in self.models.items():
            try:
                # Use accuracy for cross-validation
                cv_scores = cross_val_score(model, X, y, cv=cv_folds, scoring='accuracy')
                cv_results[name] = {
                    'mean_accuracy': cv_scores.mean(),
                    'std_accuracy': cv_scores.std(),
                    'all_scores': cv_scores
                }
                print(colored(f"  {name}: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}", "yellow"))
            except Exception as e:
                print(colored(f"  {name}: CV failed - {e}", "red"))

        self.cv_results = cv_results
        return cv_results

    def split_data(self, X, y, test_size=0.2):
        print("  Splitting data into train and test sets...")
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=test_size, random_state=42, stratify=y
        )
        print(colored(f"  Data split complete. Train: {self.X_train.shape[0]}, Test: {self.X_test.shape[0]}", "green"))

    def train_and_evaluate_all(self):
        print("\n--- Training & Evaluating Models ---")
        if self.X_train is None:
            print(colored("Error: Data has not been split yet.", "red"))
            return

        for name, model in self.models.items():
            print(f"\n  Training {name}...")
            try:
                model.fit(self.X_train, self.y_train)
                y_pred = model.predict(self.X_test)

                # Comprehensive metrics
                accuracy = accuracy_score(self.y_test, y_pred)
                precision = precision_score(self.y_test, y_pred, zero_division=0)
                recall = recall_score(self.y_test, y_pred, zero_division=0)
                f1 = f1_score(self.y_test, y_pred, zero_division=0)
                report = classification_report(self.y_test, y_pred, zero_division=0)

                self.results[name] = {
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1_score': f1,
                    'report': report
                }
                self.trained_models[name] = model

                print(colored(f"  ✅ {name} Results:", "green"))
                print(f"    Accuracy:  {accuracy * 100:.2f}%")
                print(f"    Precision: {precision * 100:.2f}%")
                print(f"    Recall:    {recall * 100:.2f}%")
                print(f"    F1-Score:  {f1 * 100:.2f}%")

            except Exception as e:
                print(colored(f"  ❌ {name} training failed: {e}", "red"))

    def plot_all_confusion_matrices(self):
        print("\n--- Generating Confusion Matrices ---")
        num_models = len(self.trained_models)
        if num_models == 0:
            print("No trained models available for plotting.")
            return

        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        axes = axes.flat

        for i, (name, model) in enumerate(self.trained_models.items()):
            if i < len(axes):
                ax = axes[i]
                ConfusionMatrixDisplay.from_estimator(
                    model, self.X_test, self.y_test,
                    ax=ax, cmap=plt.cm.Blues, colorbar=False
                )
                ax.set_title(f'{name}\nAccuracy: {self.results[name]["accuracy"]*100:.1f}%')

        # Hide empty subplots
        for i in range(len(self.trained_models), len(axes)):
            axes[i].set_visible(False)

        plt.tight_layout()
        plt.show(block=False)
        print("  Confusion matrices displayed.")

    def plot_cv_comparison(self):
        """Plot cross-validation results comparison"""
        if not self.cv_results:
            print("No cross-validation results to plot.")
            return

        names = list(self.cv_results.keys())
        means = [self.cv_results[name]['mean_accuracy'] for name in names]
        stds = [self.cv_results[name]['std_accuracy'] for name in names]

        plt.figure(figsize=(10, 6))
        y_pos = np.arange(len(names))

        plt.barh(y_pos, means, xerr=stds, align='center', alpha=0.7, capsize=5)
        plt.yticks(y_pos, names)
        plt.xlabel('Accuracy')
        plt.title('Cross-Validation Results (Mean ± Std Dev)')

        # Add value labels on bars
        for i, v in enumerate(means):
            plt.text(v + 0.01, i, f'{v:.3f}', va='center')

        plt.tight_layout()
        plt.show(block=False)

    def get_best_model(self):
        """Finds the best model based on weighted score (accuracy + f1)"""
        if not self.results:
            return None, 0, None

        # Use weighted score considering both accuracy and f1-score
        def calculate_score(result):
            return (result['accuracy'] + result['f1_score']) / 2

        best_name = max(self.results, key=lambda name: calculate_score(self.results[name]))
        best_score = self.results[best_name]['accuracy']
        best_model_object = self.trained_models[best_name]

        return best_name, best_score, best_model_object

    def save_model(self, model, filepath="enhanced_heart_model.pkl"):
        """Saves the best performing model to a file."""
        joblib.dump(model, filepath)
        print(colored(f"  Model saved to {filepath}", "cyan"))

#################################################################
# 3. ENHANCED TRAINING PIPELINE
#################################################################

def print_feature_importance(model, preprocessor, original_columns):
    """Enhanced feature importance analysis"""
    print(colored("\n--- Feature Importance Analysis ---", "yellow"))

    try:
        feature_names = preprocessor.get_feature_names_out()

        if hasattr(model, 'feature_importances_'):  # Tree-based models
            importances = model.feature_importances_
            importance_type = "Feature Importance"
        elif hasattr(model, 'coef_'):  # Linear models
            importances = np.abs(model.coef_[0])
            importance_type = "Absolute Coefficient"
        else:
            print("Feature importance not available for this model type.")
            return

        # Create feature importance dataframe
        importance_df = pd.DataFrame({
            'Feature': feature_names,
            importance_type: importances
        }).sort_values(importance_type, ascending=False)

        print(f"Top 10 Most Important Features ({importance_type}):")
        print(importance_df.head(10).to_string(index=False))

        # Plot feature importance
        plt.figure(figsize=(12, 8))
        top_features = importance_df.head(10)
        plt.barh(top_features['Feature'], top_features[importance_type])
        plt.xlabel(importance_type)
        plt.title(f'Top 10 Feature Importances\n({importance_type})')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show(block=False)

    except Exception as e:
        print(f"Could not compute feature importance: {e}")

def print_final_evaluation(model_name, accuracy, results):
    """Print comprehensive final evaluation"""
    print(colored("\n" + "="*60, "green"))
    print(colored("🎯 FINAL TRAINING RESULTS", "green", attrs=['bold']))
    print(colored("="*60, "green"))
    print(f"🏆 Best Model: {model_name}")
    print(f"📊 Test Accuracy: {accuracy * 100:.2f}%")
    print(f"📈 Precision: {results['precision'] * 100:.2f}%")
    print(f"📈 Recall: {results['recall'] * 100:.2f}%")
    print(f"📈 F1-Score: {results['f1_score'] * 100:.2f}%")
    print(f"\n📋 Detailed Classification Report:")
    print(results['report'])

def run_enhanced_training_pipeline():
    """
    Enhanced training pipeline with comprehensive validation and evaluation
    """
    print(colored("--- 🚀 ENHANCED Heart Disease Prediction Pipeline (TRAIN MODE) ---", "cyan", attrs=['bold']))

    # Configuration
    DATA_FILEPATH = "heart.csv"
    TARGET_VARIABLE = "target"

    NUMERICAL_COLS_TO_SCALE = [
        'age', 'resting_blood_pressure', 'cholesterol',
        'max_heart_rate', 'oldpeak'
    ]
    CATEGORICAL_COLS_TO_ENCODE = [
        'chest_pain_type', 'resting_ecg', 'slope',
        'vessels_colored', 'thalassemia',
        'fasting_blood_sugar', 'exercise_angina', 'sex'
    ]

    # --- 1. Enhanced Preprocessing ---
    print("\n--- 1. Enhanced Data Preprocessing ---")
    processor = EnhancedDataProcessor(DATA_FILEPATH)
    processor.load_data()
    if processor.data is None:
        return None, None, None

    processor.rename_columns()
    processor.drop_duplicates()
    processor.handle_missing_values()

    # Enhanced data validation
    processor.validate_data_quality(TARGET_VARIABLE)

    print("\n--- Data Summary ---")
    print(f"Dataset shape: {processor.data.shape}")
    print(f"Features: {list(processor.data.columns)}")

    X = processor.data.drop(TARGET_VARIABLE, axis=1)
    y = processor.data[TARGET_VARIABLE]

    processor.build_preprocessor(NUMERICAL_COLS_TO_SCALE, CATEGORICAL_COLS_TO_ENCODE)
    X_processed = processor.process_training_data(X)

    if X_processed is None:
        return None, None, None

    # --- 2. Enhanced Model Training ---
    print("\n--- 2. Enhanced Model Training ---")
    trainer = EnhancedModelTrainer()

    # Perform cross-validation first
    cv_results = trainer.perform_cross_validation(X_processed, y, cv_folds=5)

    # Plot CV results
    trainer.plot_cv_comparison()

    # Then proceed with standard train/test split
    trainer.split_data(X_processed, y)
    trainer.train_and_evaluate_all()

    # --- 3. Enhanced Evaluation ---
    print("\n--- 3. Comprehensive Model Evaluation ---")
    trainer.plot_all_confusion_matrices()

    best_name, best_score, best_model = trainer.get_best_model()

    if best_model:
        # Save artifacts
        processor.save_preprocessor("enhanced_data_preprocessor.pkl")
        trainer.save_model(best_model, "enhanced_heart_model.pkl")

        # Feature importance analysis
        print_feature_importance(best_model, processor.preprocessor, X.columns)

        # Final evaluation
        print_final_evaluation(best_name, best_score, trainer.results[best_name])

        return best_model, processor, trainer
    else:
        print(colored("Training failed - no model was successfully trained!", "red"))
        return None, None, None

#################################################################
# 4. PREDICTION APP (PREDICT MODE)
#################################################################

def get_patient_data_from_user():
    """
    Prompts the user to enter data for a new patient.
    Uses the *renamed, friendly* column names.
    """
    print(colored("\n--- 🏥 Please enter new patient data ---", "yellow"))

    patient_data = {}

    try:
        # --- Numerical Features ---
        patient_data['age'] = int(input("Enter Age (e.g., 52): "))
        patient_data['resting_blood_pressure'] = int(input("Enter Resting Blood Pressure (e.g., 120): "))
        patient_data['cholesterol'] = int(input("Enter Cholesterol (e.g., 284): "))
        patient_data['max_heart_rate'] = int(input("Enter Max Heart Rate (e.g., 162): "))
        patient_data['oldpeak'] = float(input("Enter ST Depression (oldpeak, e.g., 1.5): "))

        # --- Categorical Features ---
        patient_data['sex'] = int(input("Enter Sex (1 = male; 0 = female): "))
        patient_data['chest_pain_type'] = int(input("Enter Chest Pain Type (0-3): "))
        patient_data['fasting_blood_sugar'] = int(input("Fasting Blood Sugar > 120 mg/dl (1 = true; 0 = false): "))
        patient_data['resting_ecg'] = int(input("Enter Resting ECG (0-2): "))
        patient_data['exercise_angina'] = int(input("Enter Exercise Induced Angina (1 = yes; 0 = no): "))
        patient_data['slope'] = int(input("Enter Slope of ST segment (0-2): "))
        patient_data['vessels_colored'] = int(input("Enter Vessels Colored by Flourosopy (0-4): "))
        patient_data['thalassemia'] = int(input("Enter Thalassemia (0-3): "))

        print(colored("-----------------------------------", "yellow"))
        print(colored("Data collected successfully.", "green"))
        return patient_data

    except ValueError:
        print(colored("\n[Error] Invalid input. Please enter numerical values only.", "red"))
        return None

def run_prediction_app():
    """
    Loads the saved model and preprocessor, then predicts on new user data.
    """
    print(colored("--- 🩺 Heart Disease Prediction (PREDICT MODE) ---", "cyan", attrs=['bold']))

    # --- 1. Load Model and Preprocessor ---
    try:
        model = joblib.load("enhanced_heart_model.pkl")
        preprocessor = joblib.load("enhanced_data_preprocessor.pkl")
        print(colored("✅ Model and preprocessor loaded successfully.", "green"))
    except FileNotFoundError:
        print(colored("❌ Error: Model files not found.", "red"))
        print(colored("Please run the training pipeline first to train and save the model.", "red"))
        return
    except Exception as e:
        print(colored(f"An error occurred loading files: {e}", "red"))
        return

    # --- 2. Get Data from User ---
    patient_data_dict = get_patient_data_from_user()

    if patient_data_dict is None:
        print(colored("Prediction cancelled due to invalid input.", "red"))
        return

    # --- 3. Process and Predict ---
    try:
        # Convert dictionary to DataFrame
        patient_df = pd.DataFrame([patient_data_dict])

        # Apply the preprocessor
        processed_patient_data = preprocessor.transform(patient_df)

        # Make the prediction
        prediction = model.predict(processed_patient_data)
        prediction_proba = model.predict_proba(processed_patient_data)

        # --- 4. Show Results ---
        confidence = prediction_proba[0][prediction[0]] * 100
        disease_prob = prediction_proba[0][1] * 100  # Probability of disease

        print("\n" + "="*50)
        print(colored("📈 PREDICTION RESULTS", "cyan", attrs=['bold']))
        print("="*50)

        if prediction[0] == 1:
            print(colored(f"🔴 RESULT: HIGH RISK OF HEART DISEASE", "red", attrs=['bold']))
        else:
            print(colored(f"🟢 RESULT: LOW RISK OF HEART DISEASE", "green", attrs=['bold']))

        print(f"📊 Confidence: {confidence:.2f}%")
        print(f"🎯 Probability of Heart Disease: {disease_prob:.2f}%")
        print(f"📋 Prediction Probabilities:")
        print(f"   - No Disease: {prediction_proba[0][0]*100:.2f}%")
        print(f"   - Disease:    {prediction_proba[0][1]*100:.2f}%")

        # Risk interpretation
        if disease_prob > 70:
            print(colored("   ⚠️  High risk - Consult a doctor immediately!", "red"))
        elif disease_prob > 30:
            print(colored("   ℹ️  Moderate risk - Consider medical consultation", "yellow"))
        else:
            print(colored("   ✅ Low risk - Maintain healthy lifestyle", "green"))

    except Exception as e:
        print(colored(f"\n[Error] Failed to make prediction: {e}", "red"))
        print("This may be due to a mismatch in data format or corrupted files.")

#################################################################
# 5. MAIN EXECUTION (User Choice)
#################################################################

if __name__ == "__main__":
    print(colored("="*60, "blue"))
    print(colored("         ENHANCED HEART DISEASE PREDICTION SYSTEM", "blue", attrs=['bold']))
    print(colored("="*60, "blue"))
    print("\nWhat would you like to do?")
    print(colored("  1: [TRAIN] ", "green") + "Run enhanced training pipeline (needs heart.csv)")
    print(colored("  2: [PREDICT] ", "yellow") + "Predict on a new patient (needs trained models)")
    print(colored("  3: [BOTH] ", "cyan") + "Train then predict (complete workflow)")

    choice = input("\nEnter your choice (1, 2, or 3): ")

    if choice == '1':
        best_model, processor, trainer = run_enhanced_training_pipeline()
        if best_model:
            print(colored("\n🎉 Training completed successfully! You can now use prediction mode.", "green"))

        # Keep plots open
        print("\nPress Enter to close plots and exit...")
        input()

    elif choice == '2':
        run_prediction_app()

    elif choice == '3':
        # Train first
        best_model, processor, trainer = run_enhanced_training_pipeline()
        if best_model:
            print(colored("\n" + "="*50, "green"))
            print(colored("🚀 NOW SWITCHING TO PREDICTION MODE", "green", attrs=['bold']))
            print(colored("="*50, "green"))
            # Then predict
            run_prediction_app()
        else:
            print(colored("Cannot proceed to prediction - training failed.", "red"))

    else:
        print(colored("Invalid choice. Please run the script again and enter 1, 2, or 3.", "red"))

         ENHANCED HEART DISEASE PREDICTION SYSTEM

What would you like to do?
  1: [TRAIN] Run enhanced training pipeline (needs heart.csv)
  2: [PREDICT] Predict on a new patient (needs trained models)
  3: [BOTH] Train then predict (complete workflow)

Enter your choice (1, 2, or 3): 2
--- 🩺 Heart Disease Prediction (PREDICT MODE) ---
✅ Model and preprocessor loaded successfully.

--- 🏥 Please enter new patient data ---
Enter Age (e.g., 52): 55
Enter Resting Blood Pressure (e.g., 120): 180
Enter Cholesterol (e.g., 284): 250
Enter Max Heart Rate (e.g., 162): 140
Enter ST Depression (oldpeak, e.g., 1.5): 2
Enter Sex (1 = male; 0 = female): 1
Enter Chest Pain Type (0-3): 0
Fasting Blood Sugar > 120 mg/dl (1 = true; 0 = false): 1
Enter Resting ECG (0-2): 2
Enter Exercise Induced Angina (1 = yes; 0 = no): 1
Enter Slope of ST segment (0-2): 2
Enter Vessels Colored by Flourosopy (0-4): 1
Enter Thalassemia (0-3): 2
-----------------------------------
Data collected successfully.

📈 PREDICTIO